In [1]:
package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    '../input/image-fmix/FMix-master'
]
import sys; 

for pth in package_paths:
    sys.path.append(pth)
    
# from fmix import sample_mask, make_low_freq_image, binarise_mask

In [2]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from catalyst.data.sampler import BalanceClassSampler

import fastai
from fastai.vision import *
from fastai.layers import AdaptiveConcatPool2d, Flatten, Mish
import seaborn as sns

C:\Users\wangz\anaconda3\envs\tf_gpu23-2\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        x = self.model(x)
        return x    

class CustomResNext(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        #='resnext50_32x4d',
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x
    
class CustomViT(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        ### vit
        num_features = self.model.head.in_features
        self.model.head = nn.Linear(num_features, num_classes)
    def forward(self, x):
        x = self.model(x)
        return x  
    
class NFResNext(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        #='nf_resnet50',
        num_features = self.model.head.fc.in_features
        self.model.head.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x      

In [4]:
def rand_bbox(size, lam):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

        

class CassavaDataset(Dataset):
    def __init__(self, df, 
                 img_size,
                 transforms=None, 
                 output_label=True, 
                 one_hot_label=False,
                 do_fmix=False, 
                 fmix_params={
                     'alpha': 1., 
                     'decay_power': 3., 
                     'shape': (384, 384),
                     'max_soft': True, 
                     'reformulate': False
                 },
                 do_cutmix=False,
                 cutmix_params={
                     'alpha': 1,
                 }
                ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.do_fmix = do_fmix
        self.fmix_params = fmix_params
        self.do_cutmix = do_cutmix
        self.cutmix_params = cutmix_params
        
        self.output_label = output_label
        self.one_hot_label = one_hot_label
        self.img_size = img_size
        
        if output_label == True:
            self.labels = self.df['label'].values
            #print(self.labels)
            
            if one_hot_label is True:
                self.labels = np.eye(self.df['label'].max()+1)[self.labels]
                #print(self.labels)
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.labels[index]
          
        img  = get_img("{}".format(self.df.loc[index]['path']))

        if self.transforms:
            img = self.transforms(image=img)['image']
        
        if self.do_fmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            with torch.no_grad():
                #lam, mask = sample_mask(**self.fmix_params)
                
                lam = np.clip(np.random.beta(self.fmix_params['alpha'], self.fmix_params['alpha']),0.6,0.7)
                
                # Make mask, get mean / std
                mask = make_low_freq_image(self.fmix_params['decay_power'], self.fmix_params['shape'])
                mask = binarise_mask(mask, lam, self.fmix_params['shape'], self.fmix_params['max_soft'])
    
                fmix_ix = np.random.choice(self.df.index, size=1)[0]
                fmix_img  = get_img("{}".format(self.df.iloc[fmix_ix]['path']))

                if self.transforms:
                    fmix_img = self.transforms(image=fmix_img)['image']

                mask_torch = torch.from_numpy(mask)
                
                # mix image
                img = mask_torch*img+(1.-mask_torch)*fmix_img

                #print(mask.shape)

                #assert self.output_label==True and self.one_hot_label==True

                # mix target
                rate = mask.sum()/self.img_size/self.img_size
                target = rate*target + (1.-rate)*self.labels[fmix_ix]
                #print(target, mask, img)
                #assert False
        
        if self.do_cutmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            #print(img.sum(), img.shape)
            with torch.no_grad():
                cmix_ix = np.random.choice(self.df.index, size=1)[0]
                cmix_img  = get_img("{}".format(self.df.iloc[cmix_ix]['path']))
                if self.transforms:
                    cmix_img = self.transforms(image=cmix_img)['image']
                    
                lam = np.clip(np.random.beta(self.cutmix_params['alpha'], self.cutmix_params['alpha']),0.3,0.4)
                bbx1, bby1, bbx2, bby2 = rand_bbox((self.img_size, self.img_size), lam)

                img[:, bbx1:bbx2, bby1:bby2] = cmix_img[:, bbx1:bbx2, bby1:bby2]

                rate = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (self.img_size * self.img_size))
                target = rate*target + (1.-rate)*self.labels[cmix_ix]
                
            #print('-', img.sum())
            #print(target)
            #assert False
                            
        # do label smoothing
        #print(type(img), type(target))
        if self.output_label == True:
            return img, target
        else:
            return img

In [15]:
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        #print(image_preds.shape, exam_pred.shape)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        if ((step + 1) % 1 == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    validated_accuracy = (image_preds_all==image_targets_all).mean()
    print('validation multi-class accuracy = {:.4f}'.format(validated_accuracy))
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
    return validated_accuracy

In [16]:
def create_model_from_folder_name(file):
    if 'efb4' in file:
        return CassvaImgClassifier, 'tf_efficientnet_b4_ns', 456
    elif 'vit' in file:
        return CustomViT, 'vit_base_patch16_384', 384
    elif 'nfresnet' in file:
        return object, 'nf_resnet50', 0
    else:
        return object,'',0

In [17]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb    

In [18]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, GaussianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, RandomBrightness, RandomContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize, Rotate, 
)

from albumentations.pytorch import ToTensorV2

# def get_train_transforms():
#     return Compose([
#             RandomResizedCrop(CFG['img_size'], CFG['img_size']),
#             Transpose(p=0.5),
#             Rotate(limit=(-90,90)),
#             HorizontalFlip(p=0.5),
#             VerticalFlip(p=0.5),
# #             ShiftScaleRotate(p=0.5),
# #             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
# #             RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
#             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
# #             CoarseDropout(p=0.5),
# #             Cutout(p=0.5),
#             ToTensorV2(p=1.0),
#         ], p=1.)

def get_train_transforms(img_size):
    return Compose([
            RandomResizedCrop(img_size, img_size),
            Transpose(p=0.5),
#             Rotate(limit=(-90,90)),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
#             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_train_transforms2(img_size):
    
    return Compose([
        RandomResizedCrop(height=img_size, width=img_size),
        OneOf([RandomBrightness(limit=0.1, p=1), RandomContrast(limit=0.1, p=1), RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5)]),
#         OneOf([MotionBlur(blur_limit=3), MedianBlur(blur_limit=3), GaussianBlur(blur_limit=3),], p=0.5,),
        VerticalFlip(p=0.5),
        HorizontalFlip(p=0.5),
        ShiftScaleRotate(
            shift_limit=0.2,
            scale_limit=0.2,
            rotate_limit=20,
            interpolation=cv2.INTER_LINEAR,
            border_mode=cv2.BORDER_REFLECT_101,
            p=1,
        ),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        CoarseDropout(p=0.5),
        Cutout(p=0.5),
        ToTensorV2(p=1.0),
    ], p=1.)
  
        
def get_valid_transforms(img_size):
    return Compose([
            CenterCrop(img_size, img_size, p=1.),
            Resize(img_size, img_size),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [19]:
img_size = 384
def prepare_dataloader(df, img_size, trn_idx, val_idx, train_transforms, valid_transforms, oof_rate=0,do_fmix=False, do_cutmix=False, ):
    
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    if oof_rate>0:
        train_oof = train_.sample(int(train_.shape[0]*oof_rate))
        train_oof['label'] = train_oof['oof_label']
        train_ = pd.concat([train_[~train_['image_id'].isin(train_oof.image_id.unique().tolist())], train_oof], axis=0)
        train_ = train_.sample(train_.shape[0]).reset_index(drop=True)
    
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
        
    train_ds = CassavaDataset(train_, img_size, transforms=train_transforms, output_label=True, one_hot_label=False, do_fmix=do_fmix, do_cutmix=do_cutmix)
    valid_ds = CassavaDataset(valid_, img_size, transforms=valid_transforms, output_label=True)
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=32,
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=0,
        #sampler=BalanceClassSampler(labels=train_['label'].values, mode="downsampling")
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=32,
        num_workers=0,
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

In [20]:
train = pd.read_pickle('train_with_data2019.pkl')

In [26]:
path = './'
valid= []

for f in os.listdir(path):
    if (os.path.isdir(f"{path}{f}")) and 'baseline' in f:
        print(f)
        MODEL, model_arch, img_size = create_model_from_folder_name(f)
        if MODEL == object:
            continue
        
        seed_everything(719)
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=719).split(np.arange(train.shape[0]), train.label.values)
        try:
        
            for fold, (trn_idx, val_idx) in enumerate(folds):

                train_transforms=get_train_transforms(img_size)
                valid_transforms=get_valid_transforms(img_size)

                train_loader, val_loader = prepare_dataloader(train, img_size, trn_idx, val_idx, oof_rate=0, train_transforms=train_transforms, valid_transforms=valid_transforms)

        #         device = torch.device(CFG['device'])
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                if torch.cuda.is_available():
                    map_location=lambda storage, loc: storage.cuda()
                else:
                    map_location='cpu'
                print('device ', device)

                model = MODEL(model_arch, train.label.nunique(), pretrained=True).to(device)    
                model.load_state_dict(torch.load('{}/{}_fold_{}'.format(f, model_arch, fold)))
                loss_fn = nn.CrossEntropyLoss().to(device)
                with torch.no_grad():
                    valid_accuracy = valid_one_epoch(0, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)
        #         torch.save(model.cnn_model.state_dict(),'{}/cnn_model_fold_{}_{}'.format(CFG['model_path'], fold, CFG['tag']))
                del model, train_loader, val_loader
                torch.cuda.empty_cache()
                valid.append({'folder':f, 'fold':fold, 'val_accuracy':valid_accuracy})
        except Exception as e:
            print('exception', f, e.__str__())
            continue
        

baseline_pytorch_efb4
device  cuda


epoch 0 loss: 0.4110: 100%|██████████| 169/169 [01:15<00:00,  2.25it/s]


validation multi-class accuracy = 0.8969
device  cuda


epoch 0 loss: 0.3929: 100%|██████████| 169/169 [01:15<00:00,  2.25it/s]


validation multi-class accuracy = 0.8979
device  cuda


...

validation multi-class accuracy = 0.9291
device  cuda


epoch 0 loss: 0.3095: 100%|██████████| 169/169 [01:49<00:00,  1.54it/s]


validation multi-class accuracy = 0.9164
device  cuda


epoch 0 loss: 0.2883: 100%|██████████| 169/169 [01:49<00:00,  1.54it/s]

validation multi-class accuracy = 0.9245


In [28]:
valid_df = pd.DataFrame(valid)

In [33]:
valid_df.groupby('folder')['val_accuracy'].agg('mean')

folder
baseline_pytorch_efb4                                         0.898205
baseline_pytorch_efb4_2019_16batch_biloss_456                 0.893617
baseline_pytorch_efb4_2019_16batch_biloss_456_cutmix          0.893802
baseline_pytorch_efb4_2019_16batch_biloss_456_cutmix_30oof    0.895578
baseline_pytorch_efb4_2019_16batch_sml2_456_cutmix            0.895467
baseline_pytorch_efb4_2019_16batch_sml2_456_cutmix_30oof      0.895208
baseline_pytorch_efb4_clean_train_v1                          0.880185
baseline_pytorch_efb4_clean_train_v2                          0.898612
baseline_pytorch_efb4_clean_train_v3                          0.893432
baseline_pytorch_efb4_v13                                     0.895763
baseline_pytorch_efb4_v14                                     0.894912
baseline_pytorch_vit                                          0.906124
baseline_pytorch_vit_2019_16batch_biloss_456                  0.892581
baseline_pytorch_vit_v3                                       0.889991